In [ ]:
#install sentence_transformers
#pip install sentence_transformers

In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt 
from sklearn.neural_network import MLPClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
#Read the data
disastertweets=pd.read_csv("../input/nlp-getting-started/train.csv")
disastertweets.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [27]:
#assign text data to a pandas series textvector
#assign disaster resutls to target (responds)
textvector=disastertweets["text"]
target=disastertweets["target"]
textvector.head()

0    Our Deeds are the Reason of this #earthquake M...
1               Forest fire near La Ronge Sask. Canada
2    All residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    Just got sent this photo from Ruby #Alaska as ...
Name: text, dtype: object

In [28]:
#use BERT to convert text to feature matrix
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(textvector, show_progress_bar=True)

Batches:   0%|          | 0/238 [00:00<?, ?it/s]

In [29]:
x_train_embeddings=np.array(embeddings)
x_train_embeddings.shape

(7613, 768)

In [30]:
#Read the test data
disastertweets_test=pd.read_csv("../input/nlp-getting-started/test.csv")
disastertweets_test.head()
#convert test text to a test matrix
text_test = disastertweets_test["text"]
x_test_embeddings = model.encode(text_test, show_progress_bar=True)

Batches:   0%|          | 0/102 [00:00<?, ?it/s]

In [ ]:
x_test_embeddings=np.array(x_test_embeddings)
x_test_embeddings.shape

In [ ]:
#train a logistic classifier
from sklearn.linear_model import LogisticRegression
Logistclf = LogisticRegression(penalty="l2", max_iter=1000).fit(x_train_embeddings, target)

In [ ]:
#predict test data with SVM classifier
Logistpredict = Logistclf.predict(x_test_embeddings)
Logistpredict[0:10]

In [ ]:
#Read the sample_submission
sample_submission=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_submission["target"]=Logistpredict
#write the reults
sample_submission.to_csv("./BERT_Logist.csv",index=False)
#results report
#Use BERT to convert features from text and use logistic classifier to classify target. 
#F1=0.8075

In [ ]:
#train a naive Bayes classifier
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB().fit(x_train_embeddings, target)

In [ ]:
#predict test data
predict = clf.predict(x_test_embeddings)
predict[0:10]

In [ ]:
#Read the sample_submission
sample_submission=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_submission["target"]=predict
#write the reults
sample_submission.to_csv("./BERT_naiveBayes.csv",index=False)
#results report
#Use BERT to convert features from text and use naive Bayes classifier to classify target. 
#F1=0.7440

In [ ]:
#train a support vector machine (SVM) classifier
from sklearn.linear_model import SGDClassifier
SVMclf = SGDClassifier(loss="log",penalty="l2",alpha=1e-3).fit(x_train_embeddings, target)

In [ ]:
#predict test data with SVM classifier
SVMpredict = SVMclf.predict(x_test_embeddings)
SVMpredict[0:10]

In [ ]:
#Read the sample_submission
sample_submission=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_submission["target"]=SVMpredict
#write the reults
sample_submission.to_csv("./BERT_SVM.csv",index=False)
#results report
#Use BERT to convert features from text and use SVM to classify target. 
#F1=0.81580

In [ ]:
xgbr = xgb.XGBClassifier(max_depth=4,eta=0.6,objective='binary:logistic',reg_lambda=2)
xgbr.fit(x_train_embeddings, target)
score = xgbr.score(x_train_embeddings, target)  
print("Training score: ", score)

In [ ]:
scores = cross_val_score(xgbr, x_train_embeddings, target,cv=7)
print("Mean cross-validation score: %.2f" % scores.mean())

In [ ]:
xgbpredict = xgbr.predict(x_test_embeddings)
xgbpredict[0:10]

In [ ]:
#Read the sample_submission
sample_submission=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_submission["target"]=xgbpredict
#write the reults
sample_submission.to_csv("./BERT_xgboost.csv",index=False)
#results report
#Use BERT to convert features from text and use xgboost to classify target. 
#parameter setting:(max_depth=6,eta=0.6,objective='binary:logistic',reg_lambda=2)
#Training score:  0.865493235255484
#Mean cross-validation score: 0.69
#Test F1=0.80661

In [ ]:
#randomforest model
clf = RandomForestClassifier()
clf.fit(x_train_embeddings, target)
#predict test data
rf_predict = clf.predict(x_test_embeddings)

In [ ]:
#Read the sample_submission
sample_submission=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_submission["target"]=rf_predict
#write the reults
sample_submission.to_csv("./BERT_forest.csv",index=False)
#results report
#Use BERT to convert features from text and use random.forest to classify target. 
#Test F1=0.81366

In [ ]:
#neural network from sklearn
clf = MLPClassifier(solver='lbfgs', alpha=1e-3, random_state=1,max_iter=2000)
clf.fit(x_train_embeddings, target)

In [ ]:
sklearn_nn_predict=clf.predict(x_test_embeddings)

In [ ]:
#Read the sample_submission
sample_submission=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_submission["target"]=sklearn_nn_predict
#write the reults
sample_submission.to_csv("./BERT_sklearn_nn.csv",index=False)

In [ ]:
class traindata(Dataset):
    def __init__(self, X, y):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            self.X = torch.from_numpy(X)
            self.y = torch.Tensor(y)
        else:
            self.X = X
            self.y = y
        self.y = self.y.view(-1,1)
    def __getitem__(self, i):
        return self.X[i,], self.y[i]
    def __len__(self):
        return len(self.y)
    
class DisasterNN(nn.Module):
    def __init__(self,D_in,H,D_out):
        super().__init__()
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,D_out)
    
    def forward(self,x):
        x=torch.sigmoid(self.linear1(x))
        x=torch.sigmoid(self.linear2(x))
        return x

def trainNN(trainset,modelNN,criterion,optimizer, epochs=50):
    cost=[]
    
    for epoch in range(epochs):
        total=0
        
        for X,y in trainloader:
            optimizer.zero_grad()
            yhat=modelNN(X)
            loss=criterion(yhat,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total+=loss.item()
        
        cost.append(total)
    return cost  

In [ ]:
modelNN=DisasterNN(x_train_embeddings.shape[1],4,1)
criterion=nn.BCELoss()
optimizer=torch.optim.SGD(modelNN.parameters(),lr=0.1)
trainset=traindata(x_train_embeddings,np.array(target))
trainloader=DataLoader(dataset=trainset,batch_size=10)
#optimizer=torch.optim.Adam(modelNN.parameters(),lr=0.1,weight_decay=1e-1)
COST=trainNN(trainset,modelNN,criterion,optimizer,epochs=2000)

In [ ]:
print(sum((modelNN(torch.tensor(x_train_embeddings))>0.5).int().view(-1).numpy()==target)/len(target))
plt.plot(COST)

In [ ]:
nnpredict=(modelNN(torch.tensor(x_test_embeddings))>0.5).int().view(-1).numpy()
print(nnpredict[0:10])
sum(nnpredict)

In [ ]:
#Read the sample_submission
sample_submission=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_submission["target"]=nnpredict
#write the reults
sample_submission.to_csv("./BERT_nn.csv",index=False)
#results report
#Use BERT to convert features from text and use neural network to classify target. 
#Test F1=0.788